This Notebook is Prepared by Abdelrahman Hamza - 1200020
The purpose of the model we're building is the classification of Handwritten Arabic Letters (28) using a CNN.

The first code segment is used to upload the input data (Training & Testing Data). The data should be uploaded as a zipped folder consisting of 2 folders, train & test where each folder contains the train & test images named with the following format: Id_{Image Id}_label_{label Id}.jpg

In [1]:
#@title Upload zipped folder containing 2 directories: Test & Train (Images only)
from google.colab import files
import zipfile
import os
import shutil

uploaded = files.upload()

uploaded_zip_file_name = list(uploaded.keys())[0]

zip_destination_directory = '/content/'

shutil.move(uploaded_zip_file_name, os.path.join(zip_destination_directory, uploaded_zip_file_name))

moved_zip_file_path = os.path.join(zip_destination_directory, uploaded_zip_file_name)

extraction_path = '/content/'

os.makedirs(extraction_path, exist_ok=True)


with zipfile.ZipFile(moved_zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

extracted_contents = os.listdir(extraction_path)
print(f"Contents of the extraction directory: {extracted_contents}")


Saving trainAndTest.zip to trainAndTest.zip
Contents of the extraction directory: ['.config', 'trainAndTest.zip', 'train', 'test', 'sample_data']


The next segment handles formatting the data read by the first part in a way that is readable to tensorflow.keras
where it would split the images into subdirectory based on thier label which is provided via the file name.

In [2]:
#@title Organize Images in subdirectory format
import os
import shutil
import re

def organize_images(source_directory, destination_directory):
    # Create the destination directory if it doesn't exist
    os.makedirs(destination_directory, exist_ok=True)

    # List all files in the source directory
    all_files = os.listdir(source_directory)

    # Filter only the image files (assuming they have a common extension like '.png')
    image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Define a regular expression pattern to extract label
    pattern = re.compile(r"id_\d+_label_(\d+)\.png")

    # Organize images into subdirectories based on labels
    for image_file in image_files:
        match = pattern.match(image_file)
        if match:
            label = match.group(1)
            label_directory = os.path.join(destination_directory, f"class_{label}")
            os.makedirs(label_directory, exist_ok=True)

            # Move the image file to the corresponding label subdirectory
            source_path = os.path.join(source_directory, image_file)
            destination_path = os.path.join(label_directory, image_file)
            shutil.move(source_path, destination_path)

# Organize train images
train_source_directory = '/content/train'
train_destination_directory = '/content/organized_train'
organize_images(train_source_directory, train_destination_directory)

# Organize test images
test_source_directory = '/content/test'
test_destination_directory = '/content/organized_test'
organize_images(test_source_directory, test_destination_directory)


This next segment is for the required Imports.

In [3]:
#@title Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

The next two Segments are used to Load Data into tensorflow/keras.
the first segment uploads the data as it is (only normalizes the pixels)
the send segment uploads the data and does data augmentation on it.

Choosing which segment to run depends on the part that we're doing.


---


we can also modify some parameters in these blocks such as Batch Size, Input Image desired size & learning rate.

In [4]:
#@title Load Data to Tensorflow
batchSize =50
learningRate = 0.001
size = (32,32)
train = tf.keras.utils.image_dataset_from_directory('/content/organized_train', batch_size=batchSize, color_mode='grayscale', image_size=size )
train = train.map(lambda x,y: (x/255, y))

test = tf.keras.utils.image_dataset_from_directory('/content/organized_test' ,color_mode='grayscale', image_size= size)
test = test.map(lambda x,y: (x/255, y))

Found 13440 files belonging to 28 classes.
Found 3360 files belonging to 28 classes.


In [ ]:
#@title Load Data to Tensorflow (Data Augmentation)

batchSize = 50
learningRate = 0.0001
size = (32, 32)
sizePreTrained = (28,28)
datagen = ImageDataGenerator(
    rotation_range=5,
    shear_range=15,
    zoom_range=0.2,
    rescale=1./255
    )

train = datagen.flow_from_directory(
    directory='/content/organized_train',
    class_mode="sparse",
    target_size= sizePreTrained,
    color_mode='grayscale',
    batch_size = batchSize
)

test = tf.keras.utils.image_dataset_from_directory(
    '/content/organized_test',
    image_size=sizePreTrained,
    color_mode='grayscale'
).map(lambda x, y: (x / 255, y))


Found 13440 images belonging to 28 classes.
Found 3360 files belonging to 28 classes.


The next three segments are used to build the CNN.

---
1. The first segment builds CNN-14 (Link). which was chosen due to High accuracy as well as Model simplicity and low number of trainable parameter.

2. The 2nd segment defines a Ready Model (DenseNet121)

3. The 3rd segment asks to upload a pre-trained CNN to do transfer learning (takes a .h5 file which will be attached with the submission)

We will run one of the next 3 segments depending on which task we're doing.


In [5]:
#@title Build CNN
num_classes = 28

input_size = (32,32,1)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_size, padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu' , padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu' , padding='same'))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='relu' , padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu' , padding='same'))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=learningRate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                   

In [ ]:
#@title Build CNN (Ready Model)
from tensorflow.keras.applications import DenseNet121

model = DenseNet121(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(32,32,1),
    pooling=None,
    classes=28,
    classifier_activation="softmax",
)

model.compile(optimizer=Adam(learning_rate=learningRate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
#@title Load Pre-Trained Model
from tensorflow.keras.layers import Input, Dense, Flatten, InputLayer
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

num_classes = 28

modelFile = files.upload()
filename = list(modelFile.keys())[0]
base_model = load_model(filename)

base_model.layers[0] = InputLayer(input_shape=(28, 28, 1))

base_model.layers[-1] = Dense(28, activation='softmax')

# i = 0
# for layer in base_model.layers:
#   layer.trainable = False
#   i+=1
#   if i == 4: break

model = base_model

for layer in model.layers:
    layer.trainable = True

# Compile the new model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


The next segment trains the model. It also prints the logs to show the progression of each model as it trains.

We can define the number of Epochs as a parameter to the fit function in this code segment.

In [ ]:
#@title Train Model
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=100, batch_size=batchSize, validation_data=test, callbacks=[tensorboard_callback])

Epoch 1/100
269/269 [==============================] - 12s 15ms/step - loss: 3.1273 - accuracy: 0.0676 - val_loss: 2.4507 - val_accuracy: 0.1542
Epoch 2/100
269/269 [==============================] - 4s 14ms/step - loss: 1.9577 - accuracy: 0.2847 - val_loss: 1.2926 - val_accuracy: 0.4664
Epoch 3/100
269/269 [==============================] - 3s 12ms/step - loss: 1.3354 - accuracy: 0.4545 - val_loss: 1.0530 - val_accuracy: 0.5622
Epoch 4/100
269/269 [==============================] - 4s 15ms/step - loss: 0.9998 - accuracy: 0.5968 - val_loss: 0.6510 - val_accuracy: 0.7634
Epoch 5/100
269/269 [==============================] - 4s 14ms/step - loss: 0.6803 - accuracy: 0.7504 - val_loss: 0.4794 - val_accuracy: 0.8417
Epoch 6/100
269/269 [==============================] - 3s 11ms/step - loss: 0.5326 - accuracy: 0.8193 - val_loss: 0.3628 - val_accuracy: 0.8753
Epoch 7/100
269/269 [==============================] - 3s 11ms/step - loss: 0.3977 - accuracy: 0.8740 - val_loss: 0.2678 - val_accuracy

The next segment is used to Plot the Accuracy & Loss for both training & testing data.

In [ ]:
#@title Plot Performace
import numpy as np
from matplotlib import pyplot as plt

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='training loss')
plt.plot(hist.history['val_loss'], color='orange', label='validation loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='train accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='validation accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()